# Assignment: 1NN против RandomForest


В этом задании будет использоваться датасет `digits` из `sklearn.datasets`. Оставьте _последние_ 25% объектов для контроля качества, разделив `X` и `y` на `X_train`, `y_train` и `X_test`, `y_test`.

Целью задания будет реализовать самый простой метрический классификатор — метод ближайшего соседа, а также сравнить качество работы реализованного вами `1NN` с `RandomForestClassifier` из `sklearn` на 1000 деревьях.

In [55]:
from sklearn.datasets import load_digits

digits = load_digits()
data = digits.data
target = digits.target

In [56]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.25, shuffle=False)

In [57]:
def write_answer(filename, value):
    with open(filename, 'w') as fout:
        fout.write(str(value))

## Задание 1

Реализуйте самостоятельно метод одного ближайшего соседа с евклидовой метрикой для задачи классификации. Можно не извлекать корень из суммы квадратов отклонений, т.к. корень — монотонное преобразование и не влияет на результат работы алгоритма.

Никакой дополнительной работы с признаками в этом задании делать не нужно — мы еще успеем этим заняться в других курсах. Ваша реализация может быть устроена следующим образом: можно для каждого классифицируемого объекта составлять список пар (расстояние до точки из обучающей выборки, метка класса в этой точке), затем сортировать этот список (по умолчанию сортировка будет сначала по первому элементу пары, затем по второму), а затем брать первый элемент (с наименьшим расстоянием).

Сортировка массива длиной N требует порядка $N log N$ сравнений (более строго говоря, она работает за O(N log N)). Подумайте, как можно легко улучшить получившееся время работы. Кроме простого способа найти ближайший объект всего за N сравнений, можно попробовать придумать, как разбить пространство признаков на части и сделать структуру данных, которая позволит быстро искать соседей каждой точки. 

За выбор метода поиска ближайших соседей в `KNeigborsClassifier` из `sklearn` отвечает параметр `algorithm` — если у вас уже есть некоторый бэкграунд в алгоритмах и структурах данных, вам может быть интересно познакомиться со структурами данных `ball tree` и `kd tree`.

Доля ошибок, допускаемых 1NN на тестовой выборке, — **ответ в задании 1**.

In [58]:
import numpy as np

In [59]:
y_pred = np.zeros(X_test.shape[0])

for test_index, test_obj in enumerate(X_test):
    
    distance_value = {}
    for train_index, train_obj in enumerate(X_train):
        
        distance = np.linalg.norm(train_obj - test_obj)
        distance_value[distance] = y_train[train_index]
        
    pred = None
    for key in sorted(distance_value):
        pred = distance_value[key]
        break
        
    y_pred[test_index] = pred

In [60]:
knn_ans = np.sum(y_test != y_pred)/y_pred.shape[0]
print(knn_ans)

0.03777777777777778


In [61]:
write_answer("knn_vs_rf_ans1.txt", knn_ans)

## Задание 2

Теперь обучите на обучающей выборке `RandomForestClassifier(n_estimators=1000)` из `sklearn`. Сделайте прогнозы на тестовой выборке и оцените долю ошибок классификации на ней. Эта доля — **ответ в задании 2**. 

In [63]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=1000)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)

rf_ans = np.sum(y_test != y_pred)/y_pred.shape[0]
print(rf_ans)

0.06444444444444444


In [64]:
write_answer("knn_vs_rf_ans2.txt", rf_ans)

Обратите внимание на то, как соотносится качество работы случайного леса с качеством работы, пожалуй, одного из самых простых методов — `1NN`. Такое различие — особенность данного датасета, но нужно всегда помнить, что такая ситуация тоже может иметь место, и не забывать про простые методы.